In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import seaborn as sns
sns.set(color_codes=True)

# pip install tabula-py
import tabula


In [2]:
df_pdf = tabula.read_pdf('kcp-mag-2021.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3,]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                    )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,Код,Направление подготовки\rНаправленность (профил...,Дополнительная информация,Контрольные цифры приема,Грант\r(за счет средств\rАкадемии),По договорам об оказании платных образовательн...,NaN,NaN,NaN,NaN
1,NaN,NaN,Общие условия,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,очное,заочное,очно-\rзаочное,очное,заочное,очно-заочное,NaN,NaN
3,1,2,3,5,6,7,NaN,8,9,10
4,УСЛОВИЯ ПОСТУПЛЕНИЯ:\rФедеральное государствен...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_mos = df[ df[1].str.contains('по программе|по совокупности', na=False) ].reset_index()
df_mos.tail()

print('дубликаты: {0}' .format( df_mos[[0,1]].duplicated().sum() ))

df_mos = df_mos[[0,1]].copy().drop_duplicates()
df_mos

дубликаты: 0


,0,1
0,09.04.03,Прикладная информатика\r- по программе магистр...
1,09.04.03,Прикладная информатика\r- по программе магистр...
2,37.04.01,Психология\r- по программе магистратуры в пред...
3,37.04.01,Психология\r- по программе магистратуры в пред...
4,38.04.01,Экономика\r- по программе магистратуры в преде...
...,...,...
136,42.04.05,Медиакоммуникации\r- по программе магистратуры...
137,42.04.02,Журналистика\r- по программе магистратуры в пр...
138,46.04.01,История\r- по программе магистратуры в предела...
139,46.04.01,История\r- по программе магистратуры в предела...


In [4]:
df_one = df_mos[df_mos[1].str.contains('по программе магистратуры')]
df_one.head(3)

,0,1
0,09.04.03,Прикладная информатика\r- по программе магистр...
1,09.04.03,Прикладная информатика\r- по программе магистр...
2,37.04.01,Психология\r- по программе магистратуры в пред...


In [5]:
df_multi = df_mos[df_mos[1].str.contains('по совокупности')]
df_multi.head(3)

,0,1
33,38.04.02,Менеджмент\r- по совокупности программ магистр...
35,38.04.02,Менеджмент\r- по совокупности программ магистр...
65,38.04.04,Государственное и муниципальное управление\r- ...


In [6]:
df_tmp = pd.DataFrame([])
patern_1= "\(([ а-яa-zА-ЯA-Z0-9-:]*)\)"
patern_2 = "\([ а-яa-zА-ЯA-Z0-9-: ]*\)"
for (i, x) in zip(df_one.iloc[:, 0], df_one.iloc[:, 1]):
    dic = {}
    dic.update({0: i})
    part_1 = x.split('\r',1)[0]
    part_2 = x.split('\r',1)[1]
    tmp_1 = part_1
    dic.update({1: tmp_1})
    tmp = part_2.replace('\r'," ").split(':',1)[1]
    tmp_2 = tmp.replace(re.findall(patern_2, tmp )[-1],'').lstrip().rstrip()
    dic.update({2: tmp_2})
    tmp = part_2.replace('\r'," ")
    tmp_3 = re.findall(patern_1, tmp )[-1].lstrip().rstrip()
    dic.update({3: tmp_3})
    
    df_tmp_dict = pd.DataFrame(dic, index=[0])
    df_tmp = df_tmp.append(df_tmp_dict,ignore_index=True)
df_tmp

,0,1,2,3
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН
3,37.04.01,Психология,Психология кризисных состояний и клиническая п...,ИОН
4,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ
...,...,...,...,...
130,42.04.05,Медиакоммуникации,Цифровые коммуникации и новые медиа,ИОН
131,42.04.02,Журналистика,Журналистика и медиатехнологии,ИГСУ
132,46.04.01,История,Социально-экономическая история России и мира,ИОН
133,46.04.01,История,Россия и Франция: историко-культурные связи,ИОН


In [7]:
def program_name(patern, tmp):
    tmp = tmp.replace('\r'," ").split(':',1)[1]
    #tmp = tmp.replace('  '," ")
    tmp = tmp.replace(re.findall(patern, tmp )[-1],'')
    " ".join(tmp.split())
    tmp = tmp.strip()
    return tmp

def facult_name(patern, tmp):
    tmp = tmp.replace('\r'," ")
    tmp = re.findall(patern, tmp )[-1].strip()
    return tmp

df_one_mos = pd.DataFrame([])
df_one_mos['код'] = df_one.iloc[:, 0]
df_one_mos['стандарт'] = df_one.iloc[:,1].map(lambda x: x.split('\r',1)[0])
df_one_mos['программа'] =  df_one.iloc[:,1].map(lambda x: program_name("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x))
df_one_mos['подразделение'] =  df_one.iloc[:,1].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
df_one_mos['программа'].map(lambda x: re.sub('[ё]', 'е', x))

df_one_mos

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН
3,37.04.01,Психология,Психология кризисных состояний и клиническая п...,ИОН
4,38.04.01,Экономика,Экономика и финансы,ЭФ ЭМИТ
...,...,...,...,...
136,42.04.05,Медиакоммуникации,Цифровые коммуникации и новые медиа,ИОН
137,42.04.02,Журналистика,Журналистика и медиатехнологии,ИГСУ
138,46.04.01,История,Социально-экономическая история России и мира,ИОН
139,46.04.01,История,Россия и Франция: историко-культурные связи,ИОН


In [8]:
df_one_mos[df_one_mos['код']=='38.04.08']

,код,стандарт,программа,подразделение
98,38.04.08,Финансы и кредит,Прикладные корпоративные финансы,ВШФМ
99,38.04.08,Финансы и кредит,Денежно-кредитное и финансовое регулирование э...,ФФБ
100,38.04.08,Финансы и кредит,Регулирование финансовых рынков и институтов,ИГСУ
101,38.04.08,Финансы и кредит,Финансы и инвестиции,ИФУР


In [9]:
df_multi.head()

,0,1
33,38.04.02,Менеджмент\r- по совокупности программ магистр...
35,38.04.02,Менеджмент\r- по совокупности программ магистр...
65,38.04.04,Государственное и муниципальное управление\r- ...
104,40.04.01,Юриспруденция\r- по совокупности программ маги...
105,40.04.01,Юриспруденция\r- по совокупности программ маги...


In [10]:
def multi_transform(tmp):
    tmp_3 = tmp.iloc[1].split(':\r',1)[1]
    tmp_3 = " ".join(tmp_3.split())
    tmp_3 = re.split('\d.',tmp_3)[1:]
    tmp_3 = np.char.replace(tmp_3,'\r',' ')
    tmp_3 = np.char.replace(tmp_3,'.','')
    dic = {1: tmp.iloc[0],
           2: tmp.iloc[1].split('\r',1)[0],
           3: tmp_3
          }
    df_tmp = pd.DataFrame(dic)
    df_tmp[4] = df_tmp[3].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
    df_tmp[3] = df_tmp[3].map(lambda x: x.replace(re.findall("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x )[-1],'').strip() )
    return df_tmp
tmp = [ multi_transform(df_multi.loc[idx]) for idx in df_multi.index.values]
df_multi_mos = pd.concat(tmp, ignore_index=True)
df_multi_mos.columns = df_one_mos.columns.values
display(df_multi_mos.head(3))
display(df_multi_mos.tail(3))

,код,стандарт,программа,подразделение
0,38.04.02,Менеджмент,Управление рекламным и медиабизнесом,ИОМ
1,38.04.02,Менеджмент,Цифровая трансформация в бизнесе,ИОМ
2,38.04.02,Менеджмент,Управление производственными системами,ИОМ


,код,стандарт,программа,подразделение
25,40.04.01,Юриспруденция,Международная безопасность и права человека,ЮФ ИПНБ
26,40.04.01,Юриспруденция,Международное транспортное право,ЮФ ИПНБ
27,40.04.01,Юриспруденция,Правовое обеспечение государственной службы ка...,ЮФ ИПНБ


In [11]:
df_multi_mos

,код,стандарт,программа,подразделение
0,38.04.02,Менеджмент,Управление рекламным и медиабизнесом,ИОМ
1,38.04.02,Менеджмент,Цифровая трансформация в бизнесе,ИОМ
2,38.04.02,Менеджмент,Управление производственными системами,ИОМ
3,38.04.02,Менеджмент,Технологическое предпринимательство,ИОМ
4,38.04.04,Государственное и муниципальное управление,Государственное управление и внешнеполитическа...,ФНБ ИПНБ
5,38.04.04,Государственное и муниципальное управление,Государственное управление и национальная безо...,ФНБ ИПНБ
6,40.04.01,Юриспруденция,Юрист в государственном аппарате,ЮФ ИПНБ
7,40.04.01,Юриспруденция,Юрист в уголовном судопроизводстве,ЮФ ИПНБ
8,40.04.01,Юриспруденция,"Юрист для бизнеса, власти и обеспечения прав ч...",ЮФ ИПНБ
9,40.04.01,Юриспруденция,Правовое обеспечение контрольно-надзорной деят...,ЮФ ИПНБ


In [12]:
df_moscow = pd.concat([df_one_mos, df_multi_mos], ignore_index=True)
df_moscow.drop_duplicates(inplace=True)
df_moscow.head(3)

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data ...,ФИТАД ЭМИТ
2,37.04.01,Психология,Психология управления,ИОН


In [13]:
tmp = df_moscow[df_moscow['код']=='38.04.01'].sort_values(by = 'программа',ignore_index=True)
display(tmp)

,код,стандарт,программа,подразделение
0,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ
1,38.04.01,Экономика,Бухгалтер-аналитик: корпоративная отчетность и...,ФФБ
2,38.04.01,Экономика,Государственное регулирование экономики,ИГСУ
3,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
4,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
5,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ
6,38.04.01,Экономика,Управление инвестициями и инновациями в экономике,ИГСУ
7,38.04.01,Экономика,Финансовая дипломатия,ФФБ
8,38.04.01,Экономика,Финансовый контроль и государственный аудит,ИГСУ
9,38.04.01,Экономика,Финансовый контроль и государственный аудит (р...,ИГСУ


In [14]:
tmp = df_moscow[df_moscow['код']=='38.04.01'].sort_values(by = 'программа',ignore_index=True)
display(tmp)

,код,стандарт,программа,подразделение
0,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ
1,38.04.01,Экономика,Бухгалтер-аналитик: корпоративная отчетность и...,ФФБ
2,38.04.01,Экономика,Государственное регулирование экономики,ИГСУ
3,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
4,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
5,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ
6,38.04.01,Экономика,Управление инвестициями и инновациями в экономике,ИГСУ
7,38.04.01,Экономика,Финансовая дипломатия,ФФБ
8,38.04.01,Экономика,Финансовый контроль и государственный аудит,ИГСУ
9,38.04.01,Экономика,Финансовый контроль и государственный аудит (р...,ИГСУ


In [15]:
for cod in df_moscow['код'].unique():
    tmp = df_moscow[df_moscow['код']==cod].sort_values(by = 'программа',ignore_index=True)
    with pd.option_context('display.max_colwidth', None):
        display(tmp)

,код,стандарт,программа,подразделение
0,09.04.03,Прикладная информатика,Анализ данных и искусственный интеллект/ Data Science and Artificial Intelligence,ФИТАД ЭМИТ
1,09.04.03,Прикладная информатика,Цифровые технологии в экономике,ФИТАД ЭМИТ


,код,стандарт,программа,подразделение
0,37.04.01,Психология,Психология кризисных состояний и клиническая психология,ИОН
1,37.04.01,Психология,Психология управления,ИОН


,код,стандарт,программа,подразделение
0,38.04.01,Экономика,"Банки, финансы, инвестиции",ФФБ
1,38.04.01,Экономика,Бухгалтер-аналитик: корпоративная отчетность и финансовое моделирование,ФФБ
2,38.04.01,Экономика,Государственное регулирование экономики,ИГСУ
3,38.04.01,Экономика,Корпоративная экономика,ЭФ ЭМИТ
4,38.04.01,Экономика,Поведенческая экономика,ЭФ ЭМИТ
5,38.04.01,Экономика,Системы больших данных в экономике,ЭФ ЭМИТ
6,38.04.01,Экономика,Управление инвестициями и инновациями в экономике,ИГСУ
7,38.04.01,Экономика,Финансовая дипломатия,ФФБ
8,38.04.01,Экономика,Финансовый контроль и государственный аудит,ИГСУ
9,38.04.01,Экономика,Финансовый контроль и государственный аудит (российско-французская программа),ИГСУ


,код,стандарт,программа,подразделение
0,38.04.02,Менеджмент,Digital design в менеджменте (информационно- аналитический менеджмент),ИОН
1,38.04.02,Менеджмент,Master of Social Sciences (China and Global Governance) / Магистр наук (Китай и глобальное управление). Англоязычная магистратура,ФЭСН
2,38.04.02,Менеджмент,Внутренний контроль и аудит,ВШФМ
3,38.04.02,Менеджмент,Девелопмент и управление недвижимостью,ИОМ
4,38.04.02,Менеджмент,Деловое администрирование,ИБДА
5,38.04.02,Менеджмент,Инновационный менеджмент,ВШКУ
6,38.04.02,Менеджмент,Информационный менеджмент,Школа IT-менеджмента ЭМИТ
7,38.04.02,Менеджмент,Контрактный управляющий,ИОМ
8,38.04.02,Менеджмент,Корпоративное управление и стратегическое развитие бизнеса,ВШКУ
9,38.04.02,Менеджмент,Корпоративный финансовый менеджмент,ВШФМ


,код,стандарт,программа,подразделение
0,38.04.03,Управление персоналом,Управление карьерой персонала,ИГСУ
1,38.04.03,Управление персоналом,Управление персоналом государственной службы и корпораций,ИГСУ


...

,код,стандарт,программа,подразделение
0,41.04.05,Международные отношения,Международное экономическое сотрудничество,ФММС ИУРР


,код,стандарт,программа,подразделение
0,42.04.01,Реклама и связи с общественностью,Управление публичными коммуникациями,ИОН


,код,стандарт,программа,подразделение
0,42.04.05,Медиакоммуникации,Цифровые коммуникации и новые медиа,ИОН


,код,стандарт,программа,подразделение
0,42.04.02,Журналистика,Журналистика и медиатехнологии,ИГСУ


,код,стандарт,программа,подразделение
0,46.04.01,История,Политическая и культурная история Европы (с углубленным изучением иностранного языка),ИОН
1,46.04.01,История,Россия и Франция: историко-культурные связи,ИОН
2,46.04.01,История,Социально-экономическая история России и мира,ИОН


In [16]:
df_moscow[df_moscow['программа'].duplicated()]

,код,стандарт,программа,подразделение
